In [14]:
import numpy as np
import re
import os
import pandas as pd
from termcolor import colored
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
import json
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import glob
import os.path
import numpy as np
import sys
import codecs

In [2]:
def clean(text,shrink=False):
    text=text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text=re.sub('[“"”]',' " ',text)
    retain='[^abcdefghijklmnopqrstuvwxyz!#?" ]'
    text=re.sub('[()–-]',' ',text)
    text=re.sub(retain,'',text)
    text=text.replace('?',' ? ')
    text=text.replace('#',' # ')
    text=text.replace('!',' ! ')
    text=text.split()
    Lemmatizer=WordNetLemmatizer()
    Stemmer=PorterStemmer()
    if shrink:    
        text=[Stemmer.stem(Lemmatizer.lemmatize(word, pos='v')) for word in text]
    return ' '.join(text)

In [3]:
directory='datasets/train-task2-TC.labels'
props_=open(directory).read().split('\n')[:-1]
sentences,Articles, labels=[],[],[]
IDS=[]
Spans=[]
for prop in props_:
    prop_=prop.split('\t')
    x,y=int(prop_[2]),int(prop_[3])
    Spans.append((x,y))
    sentences.append([open('datasets/train-articles/article{}.txt'.format(prop_[0])).read()[:x],open('datasets/train-articles/article{}.txt'.format(prop_[0])).read()[x:y],open('datasets/train-articles/article{}.txt'.format(prop_[0])).read()[y:]])
    Articles.append(open('datasets/train-articles/article{}.txt'.format(prop_[0])).read())
    labels.append(prop_[1])
    IDS.append(prop_[0])
    ids=int(prop_[0])

In [4]:
ind=2
def gen(ind):
    print(labels[ind])
    pre=colored(sentences[ind][0], color='green')
    sen=colored(sentences[ind][1], color='red')
    post=colored(sentences[ind][-1], color='green')
    print(pre+sen+post)
def span_read(id,span):
    return open('datasets/train-articles/article{}.txt'.format(id)).read()[span[0]:span[1]]
def freq_dict(text,Stem=False):
    freq={}
    for word in clean(text,Stem).split():
        if word in freq:
            freq[word]=freq[word]+1
        else:
            freq[word]=1
    return freq

In [5]:
df=pd.DataFrame({'Id':IDS, 'Span':Spans, 'Label':labels})
set(labels)

{'Appeal_to_Authority',
 'Appeal_to_fear-prejudice',
 'Bandwagon,Reductio_ad_hitlerum',
 'Black-and-White_Fallacy',
 'Causal_Oversimplification',
 'Doubt',
 'Exaggeration,Minimisation',
 'Flag-Waving',
 'Loaded_Language',
 'Name_Calling,Labeling',
 'Repetition',
 'Slogans',
 'Thought-terminating_Cliches',
 'Whataboutism,Straw_Men,Red_Herring'}

In [6]:
dumps=True
if(not dumps):
    Stitch=""
    for doc in os.listdir('datasets/train-articles'):
        Stitch=Stitch+open('datasets/train-articles/'+doc).read()    
    f = open("dumps/Text_stitch.txt","w")
    f.write(Stitch)
    f.close()
    corp_freq=dict()
    Doc_freqs=dict()
    corp_freq=freq_dict(Stitch)
    for id in IDS :
        txt=open('datasets/train-articles/article{}.txt'.format(id)).read()
        Doc_freqs[id]=freq_dict(txt)

    corp_freq_json = json.dumps(corp_freq)
    f = open("dumps/Corp_freq_raw.json","w")
    f.write(corp_freq_json)
    f.close()

    Doc_freqs_json = json.dumps(Doc_freqs)
    f = open("dumps/Doc_freqs_raw.json","w")
    f.write(Doc_freqs_json)
    f.close()

    corp_freq_stem=dict()
    Doc_freqs_stem=dict()
    corp_freq_stem=freq_dict(Stitch,Stem=True)
    for id in IDS :
        txt=open('datasets/train-articles/article{}.txt'.format(id)).read()
        Doc_freqs_stem[id]=freq_dict(txt,Stem=True)

    corp_freq_json_stem = json.dumps(corp_freq_stem)
    f = open("dumps/Corp_freq_stem.json","w")
    f.write(corp_freq_json_stem)
    f.close()

    Doc_freqs_json_stem = json.dumps(Doc_freqs_stem)
    f = open("dumps/Doc_freqs_stem.json","w")
    f.write(Doc_freqs_json_stem)
    f.close()

try: 
    Stitch 
except: 
    Stitch=open("dumps/Text_stitch.txt",'r').read()
try: 
    corp_freq_raw 
except: 
    corp_freq_raw=json.loads(open("dumps/Corp_freq_raw.json",'r').read())
try: 
    Doc_freqs_raw 
except: 
    Doc_freqs_raw=json.loads(open("dumps/Doc_freqs_raw.json",'r').read())
try: 
    corp_freq_stem 
except: 
    corp_freq_stem=json.loads(open("dumps/Corp_freq_stem.json",'r').read())
try: 
    Doc_freqs_stem 
except: 
    Doc_freqs_stem=json.loads(open("dumps/Doc_freqs_stem.json",'r').read())

In [7]:
def pad(val):
    t_df=df[df['Label']=='Doubt']
    t_id,t_Span=t_df['Id'],t_df['Span']
    padsent_d=[]
    for ids,spans in zip(t_id,t_Span):
        padsent_d.append(span_read(ids,(spans[0]-val,spans[1]+val)))

    t_df=df[df['Label']!='Doubt']
    t_id,t_Span=t_df['Id'],t_df['Span']
    padsent=[]
    for ids,spans in zip(t_id,t_Span):
        padsent.append(span_read(ids,(spans[0]-val,spans[1]+val)))
    return padsent_d,padsent
padsent_d,padsent=pad(0)

In [41]:
def print_res(val,wts):
    isin,notin=pad(val)
    c_,c1_,c,c1,t,t1=0,0,0,0,0,0
    for sent in isin:
        t=t+1
        cond=False
        for check in wts:
            if check in clean(sent).split():
                cond=True
                c_=c_+wts[check]
        if cond:
            c=c+1
    for sent in notin:
        t1=t1+1
        cond=False
        for check in wts:
            if check in clean(sent).split():
                cond=True
                c1_=c1_+wts[check]
        if cond:
            c1=c1+1
    print(c/t,c_/t)
    print(c1/t1,c1_/t1)
    print((c*t1)/(c1*t),(c_*t1)/(c1_*t))

In [101]:
wts={
    '?' : 2.5,
    'how' : 1,
    'when' : 1,
    'where' : 1,
    'why' :10,
    'while' : 1,
    'what' : 1,
    'cant' : 1,
}
print_res(15,wts)

0.46169354838709675 1.7580645161290323
0.09569215051932572 0.22671547760939895
4.8247797325220985 7.7544971109469065


In [102]:
wts.keys()

dict_keys(['?', 'how', 'when', 'where', 'why', 'while', 'what', 'cant'])

In [115]:
for row in df[df['Label']=='Slogans'].values:
    print(span_read(row[0],(row[1][0]-15,row[1][1]+15)))

i-Muslim group Stop Islamization of America.

They were due 
 a pro-Israel "Defeat Jihad" poster campaig
ty of justice.
BUILD THE WALL!"
Trump tweeted.
 new partner.
"Working together, we'll get it taken care of," Trump said.

 remember that liberty of conscience is not up for a vote,” he said.

Tr
th the phrase “Israel must be wiped out” emblazoned on 
enly calls for “Death to Israel” and labels the
g “David Duke, a man to believe in!”
In July 2010,
 Opposing BDS

First they came for the Jews.
And then they came for everyone else.
Jewish studen
 followers to “punch a Zionist today.” It is unclea
 light of his “punch a Zionist” tweet calling
erred to as a “sex trade on Capitol Hill.”
The American p
, this is now.
I’m no longer afraid.
When the story 
erred to as a “sex trade on Capitol Hill.”
Do you want to
ers.
I believe today is a day of vindication for the rights of immigrants.”
Alex Bastian of
rtist to spray SMASH THE STATE on his office 
, ni tontos,” (Neither lefties nor stupid).

In [112]:
df[df['Label']=='Slogans'].values

array([['111111112', (191, 221), 'Slogans'],
       ['111111112', (785, 798), 'Slogans'],
       ['111111113', (5775, 5791), 'Slogans'],
       ['111111131', (2953, 2997), 'Slogans'],
       ['111111132', (12045, 12087), 'Slogans'],
       ['698503276', (3107, 3132), 'Slogans'],
       ['701225819', (3787, 3804), 'Slogans'],
       ['701225819', (1967, 1987), 'Slogans'],
       ['701553469', (77, 143), 'Slogans'],
       ['701553469', (3455, 3476), 'Slogans'],
       ['704591553', (2765, 2780), 'Slogans'],
       ['707451080', (1629, 1656), 'Slogans'],
       ['707772906', (1363, 1385), 'Slogans'],
       ['708561738', (2291, 2318), 'Slogans'],
       ['711566593', (1381, 1442), 'Slogans'],
       ['713130996', (1783, 1798), 'Slogans'],
       ['725498022', (6105, 6131), 'Slogans'],
       ['725731328', (547, 563), 'Slogans'],
       ['728169864', (5753, 5774), 'Slogans'],
       ['728972961', (1400, 1452), 'Slogans'],
       ['729651527', (2760, 2775), 'Slogans'],
       ['729668796',

## Vectorization

In [116]:
mapping=dict()
inv_mapping=dict()
c=0
for i in set(labels):
    mapping[c]=i
    inv_mapping[i]=c
    c=c+1
label=inv_mapping.keys()

In [118]:
id_label=dict()
label_id=dict()
for i,j in zip(IDS,labels):
    if i in id_label:
        id_label[i].append(j)
    else :
        id_label[i]=[j]

In [119]:
def vectorize(p_list):
    vec=[0]*14
    for i in p_list:
        vec[inv_mapping[i]]=vec[inv_mapping[i]]+1
    return tuple(vec)
def b_vectorize(p_list):
    return tuple([int(i>0) for i in vectorize(p_list)])

data=[]
for k,v in id_label.items():
    data.append((k,)+vectorize(v))
idf=pd.DataFrame(data,columns=['ID']+list(inv_mapping.keys()))
idf['length']=idf['ID'].apply(lambda x : len(span_read(x,[0,-1])))

b_data=[]
for k,v in id_label.items():
    b_data.append((k,)+b_vectorize(v))
b_idf=pd.DataFrame(b_data,columns=['ID']+list(inv_mapping.keys()))
b_idf['length']=b_idf['ID'].apply(lambda x : len(span_read(x,[0,-1])))

In [122]:
norm_idf=idf.copy()
norm_idf['length']=norm_idf['length']/norm_idf['length'].mean()
for cat in norm_idf.columns[1:-1]:
    norm_idf[cat]=norm_idf[cat]/norm_idf['length']

In [123]:
norm_idf.head()

,ID,Loaded_Language,Slogans,"Whataboutism,Straw_Men,Red_Herring",Appeal_to_fear-prejudice,Causal_Oversimplification,Repetition,Appeal_to_Authority,Doubt,Flag-Waving,"Name_Calling,Labeling","Exaggeration,Minimisation",Black-and-White_Fallacy,"Bandwagon,Reductio_ad_hitlerum",Thought-terminating_Cliches,length,count
0,111111111,0.000000,0.000000,0.000000,9.843858,0.000000,2.460964,4.921929,2.460964,0.00000,0.000000,0.0,0.00000,0.0,0.0,1.0,19.687716
1,111111112,5.611769,3.741179,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.87059,3.741179,0.0,1.87059,0.0,0.0,1.0,16.835307
2,111111113,3.525165,0.881291,0.881291,0.000000,2.643874,0.000000,0.000000,0.000000,0.00000,1.762583,0.0,0.00000,0.0,0.0,1.0,9.694205
3,111111114,0.000000,0.000000,0.000000,2.619338,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.0,0.0,1.0,2.619338
4,111111115,6.870274,0.000000,0.000000,0.000000,0.000000,0.000000,2.290091,0.000000,0.00000,0.000000,0.0,0.00000,0.0,0.0,1.0,9.160366


## Multiplicity

In [131]:
X=list(set(labels))
for L in X:
    print(norm_idf[norm_idf[L]>0][L].describe(),end='\n\n\n')

count    309.000000
mean       6.929812
std        4.568400
min        0.410278
25%        3.521560
50%        5.965187
75%        9.297641
max       25.232400
Name: Loaded_Language, dtype: float64


count    77.000000
mean      2.076194
std       2.038275
min       0.133347
25%       0.881291
50%       1.371974
75%       2.576224
max      11.391364
Name: Slogans, dtype: float64


count    70.000000
mean      1.984891
std       1.823956
min       0.120892
25%       0.792851
50%       1.321003
75%       2.383574
max       9.865009
Name: Whataboutism,Straw_Men,Red_Herring, dtype: float64


count    155.000000
mean       2.623486
std        2.882743
min        0.133347
25%        0.964002
50%        1.642991
75%        3.073855
max       16.471006
Name: Appeal_to_fear-prejudice, dtype: float64


count    103.000000
mean       1.941928
std        1.488283
min        0.183488
25%        0.941941
50%        1.620720
75%        2.491856
max        9.946220
Name: Causal_Oversimplification, dty

In [132]:
for L in X:
    print(norm_idf[L].describe(),end='\n\n\n')

count    357.000000
mean       5.998073
std        4.864222
min        0.000000
25%        2.352037
50%        5.320429
75%        8.523222
max       25.232400
Name: Loaded_Language, dtype: float64


count    357.000000
mean       0.447807
std        1.272077
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max       11.391364
Name: Slogans, dtype: float64


count    357.000000
mean       0.389194
std        1.125871
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        9.865009
Name: Whataboutism,Straw_Men,Red_Herring, dtype: float64


count    357.000000
mean       1.139049
std        2.300100
min        0.000000
25%        0.000000
50%        0.000000
75%        1.340567
max       16.471006
Name: Appeal_to_fear-prejudice, dtype: float64


count    357.000000
mean       0.560276
std        1.187818
min        0.000000
25%        0.000000
50%        0.000000
75%        0.755326
max        9.946220
Name: Causal_Oversim

In [133]:
idf.head()

,ID,Loaded_Language,Slogans,"Whataboutism,Straw_Men,Red_Herring",Appeal_to_fear-prejudice,Causal_Oversimplification,Repetition,Appeal_to_Authority,Doubt,Flag-Waving,"Name_Calling,Labeling","Exaggeration,Minimisation",Black-and-White_Fallacy,"Bandwagon,Reductio_ad_hitlerum",Thought-terminating_Cliches,length,count
0,111111111,0,0,0,4,0,1,2,1,0,0,0,0,0,0,2332,19.687716
1,111111112,3,2,0,0,0,0,0,0,1,2,0,1,0,0,3068,16.835307
2,111111113,4,1,1,0,3,0,0,0,0,2,0,0,0,0,6512,9.694205
3,111111114,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2191,2.619338
4,111111115,3,0,0,0,0,0,1,0,0,0,0,0,0,0,2506,9.160366


In [ ]:
_x,_y=idf[X[0]],idf[X[12]]
plt.scatter(_x,_y)
plt.show()
print(pearsonr(_x,_y))
print(spearmanr(_x,_y))

In [ ]:
sns.relplot(y=X[2], x=idf.index, hue=X[11], data=idf)

In [ ]:
train=b_idf.drop(['ID','length'],axis=1)
sns.set_style("dark")
def correlation_heatmap(train):
    correlations = train.corr('spearman')

    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, center=0.0, fmt='.2f',
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70})
    plt.show();
    
correlation_heatmap(train)

In [ ]:
correlations = train.corr()
correlations

In [ ]:
for i in range(0,14):
    q=((b_idf[X[0]]&b_idf[X[i]]).mean(),)
    print(q)

In [ ]:
b_idf.columns

In [ ]:
(b_idf[X[0]]&b_idf[X[i]])

In [149]:
for n in range(14):
    print(idf[idf[X[n]]!=0]['count'].describe([0.8]))
    print(idf[idf[X[n]]==0]['count'].describe([0.8]))

count    309.000000
mean      19.540734
std       12.173004
min        0.951105
50%       16.576350
80%       29.489613
max       67.573072
Name: count, dtype: float64
count    48.000000
mean     10.121207
std       8.084756
min       1.248688
50%       7.617709
80%      16.983031
max      33.313278
Name: count, dtype: float64
count    77.000000
mean     23.639513
std      11.930715
min       5.733902
50%      21.862740
80%      32.969285
max      63.901566
Name: count, dtype: float64
count    280.000000
mean      16.798794
std       11.785776
min        0.951105
50%       14.160704
80%       24.241362
max       67.573072
Name: count, dtype: float64
count    70.000000
mean     23.869497
std      13.293343
min       6.039431
50%      21.859883
80%      33.009615
max      67.573072
Name: count, dtype: float64
count    287.000000
mean      16.909547
std       11.447546
min        0.951105
50%       14.279272
80%       24.955937
max       59.092960
Name: count, dtype: float64
count    155.

In [7]:
def getWordSpans(text):
    wordlist=[]
    def trans(text,pointer=0):
        if pointer==len(text)-1:
            return True
        else:
            while(not text[pointer].isalpha() and pointer<len(text)-1):
                pointer=pointer+1
            s=pointer
            while(text[pointer].isalpha() and pointer<len(text)-1):
                pointer=pointer+1
            wordlist.append([s,pointer])
            return trans(text,pointer)
    try:
        trans(text)
    except :
        return -1
    if(wordlist[-1][1]==wordlist[-1][0]):
        wordlist=wordlist[:-1]
    if(text[-1].isalpha()):
        wordlist[-1][1]+=1
    return wordlist

In [8]:
getWordSpans(open('datasets/dev-articles/article730081389.txt','r').read())

[[0, 6],
 [7, 10],
 [11, 21],
 [22, 26],
 [27, 29],
 [30, 34],
 [35, 40],
 [41, 45],
 [46, 52],
 [53, 61],
 [62, 66],
 [67, 73],
 [75, 84],
 [87, 93],
 [94, 106],
 [107, 115],
 [116, 124],
 [125, 128],
 [129, 139],
 [140, 144],
 [145, 147],
 [148, 151],
 [152, 156],
 [157, 162],
 [163, 164],
 [165, 168],
 [169, 176],
 [177, 181],
 [182, 185],
 [186, 192],
 [193, 201],
 [202, 206],
 [207, 210],
 [211, 218],
 [220, 223],
 [224, 226],
 [227, 234],
 [235, 239],
 [240, 244],
 [245, 249],
 [251, 257],
 [258, 265],
 [266, 270],
 [271, 274],
 [275, 283],
 [284, 290],
 [291, 293],
 [294, 305],
 [306, 310],
 [312, 323],
 [324, 332],
 [333, 337],
 [338, 345],
 [351, 354],
 [355, 362],
 [363, 370],
 [376, 380],
 [381, 387],
 [388, 395],
 [396, 401],
 [402, 406],
 [407, 415],
 [416, 418],
 [419, 423],
 [424, 432],
 [433, 438],
 [439, 445],
 [446, 451],
 [452, 462],
 [463, 465],
 [466, 467],
 [472, 476],
 [477, 479],
 [480, 484],
 [486, 489],
 [490, 497],
 [502, 506],
 [507, 510],
 [511, 518],
 [519

In [13]:
c=0
for i in os.listdir('datasets/train-articles'):
    print(len(getWordSpans(open('datasets/train-articles/'+i,'r').read())),c)
    c=c+1

550 0
754 1
335 2
438 3
478 4
443 5
433 6
901 7
420 8
239 9
910 10
649 11
901 12
542 13
672 14
1244 15
412 16
938 17
466 18
753 19
432 20
917 21
1291 22
598 23
501 24
960 25
635 26
321 27
515 28
443 29
795 30
701 31
433 32
497 33
116 34
172 35
285 36
704 37
719 38
357 39
469 40
2259 41
825 42
590 43
3830 44
360 45
594 46
551 47
2067 48
694 49
378 50
660 51
1065 52
7288 53
296 54
364 55
390 56
1029 57
1174 58
388 59
394 60
715 61
485 62
644 63
507 64
550 65
908 66
1305 67
240 68
613 69
831 70
969 71
505 72
205 73
167 74
340 75
582 76
685 77
2177 78
421 79
523 80
111 81
184 82
916 83
2465 84
238 85
1883 86
1966 87
433 88
5208 89
979 90
826 91
394 92
3180 93
1337 94
638 95
905 96
572 97
733 98
364 99
925 100
1511 101
1040 102
456 103
696 104
396 105
1739 106
420 107
1040 108
3065 109
380 110
903 111
1379 112
441 113
1320 114
1429 115
884 116
374 117
447 118
678 119
566 120
727 121
904 122
330 123
784 124
696 125
293 126
2516 127
1053 128
339 129
1048 130
672 131
811 132
2035 133
985 134
5

In [10]:
import sys 
sys.setrecursionlimit(10**4) 

In [16]:
train_folder = "datasets/train-articles" # check that the path to the datasets folder is correct, 
dev_folder = "datasets/dev-articles"     # if not adjust these variables accordingly
train_labels_file = "datasets/train-task2-TC.labels"
dev_template_labels_file = "datasets/dev-task-TC-template.out"
task_TC_output_file = "baseline-output-TC.txt"

def read_articles_from_file_list(folder_name, file_pattern="*.txt"):
    """
    Read articles from files matching patterns <file_pattern> from  
    the directory <folder_name>. 
    The content of the article is saved in the dictionary whose key
    is the id of the article (extracted from the file name).
    Each element of <sentence_list> is one line of the article.
    """
    file_list = glob.glob(os.path.join(folder_name, file_pattern))
    articles = {}
    article_id_list, sentence_id_list, sentence_list = ([], [], [])
    for filename in sorted(file_list):
        article_id = os.path.basename(filename).split(".")[0][7:]
        with codecs.open(filename, "r", encoding="utf8") as f:
            articles[article_id] = f.read()
    return articles


def read_predictions_from_file(filename):
    """
    Reader for the gold file and the template output file. 
    Return values are four arrays with article ids, labels 
    (or ? in the case of a template file), begin of a fragment, 
    end of a fragment. 
    """
    articles_id, span_starts, span_ends, gold_labels = ([], [], [], [])
    with open(filename, "r") as f:
        for row in f.readlines():
            article_id, gold_label, span_start, span_end = row.rstrip().split("\t")
            articles_id.append(article_id)
            gold_labels.append(gold_label)
            span_starts.append(span_start)
            span_ends.append(span_end)
    return articles_id, span_starts, span_ends, gold_labels


def compute_features(articles, span_starts, span_ends):
    # only one feature, the length of the span
    return np.array([ int(sp_ends)-int(sp_starts) for sp_starts, sp_ends in zip(span_starts, span_ends) ]).reshape(-1, 1)


In [17]:
### MAIN ###

# loading articles' content from *.txt files in the train folder
articles = read_articles_from_file_list(train_folder)

In [18]:
# loading gold labels, articles ids and sentence ids from files *.task-TC.labels in the train labels folder 
ref_articles_id, ref_span_starts, ref_span_ends, train_gold_labels = read_predictions_from_file(train_labels_file)
print("Loaded %d annotations from %d articles" % (len(ref_span_starts), len(set(ref_articles_id))))

Loaded 6369 annotations from 357 articles


In [19]:
# compute one feature for each fragment, i.e. the length of the fragment, and train the model
train = compute_features(articles, ref_span_starts, ref_span_ends)

In [28]:
model = LogisticRegression(penalty='l2', class_weight='balanced', solver="lbfgs")
model.fit(train, train_gold_labels)

# reading data from the development set
dev_articles = read_articles_from_file_list(dev_folder)
dev_article_ids, dev_span_starts, dev_span_ends, dev_labels = read_predictions_from_file(dev_template_labels_file)

# computing the predictions on the development set
dev = compute_features(dev_articles, dev_span_starts, dev_span_ends)
predictions = model.predict(dev)

# writing predictions to file
with open(task_TC_output_file, "w") as fout:
    for article_id, prediction, span_start, span_end in zip(dev_article_ids, predictions, dev_span_starts, dev_span_ends):
        fout.write("%s\t%s\t%s\t%s\n" % (article_id, prediction, span_start, span_end))
print("Predictions written to file " + task_TC_output_file)

/home/somesh/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Predictions written to file baseline-output-TC.txt
